In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load in data

In [ ]:
df = pd.read_csv('/kaggle/input/billboard-the-hot-100-songs/charts.csv')
df.tail(20)

## find those songs with featuring/collab

In [ ]:
df['artist_nofeatured'] = df.artist.str.replace(r" Featuring.*$","")
df['isfeaturing'] = df.artist.str.find("Featuring")
df['isfeaturing'] = df.isfeaturing.apply(lambda x: 1 if x >0 else 0)


In [ ]:
df['iscollab'] = df.artist.str.find(" x ")
df['iscollab'] = df.iscollab.apply(lambda x: 1 if x >0 else 0)
df['iscollab'] = df['iscollab'] +df.artist.str.find("&").apply(lambda x: 1 if x >0 else 0)

## counting all the artists on the board(collabration counted as a single artist)

In [ ]:
len(df.artist_nofeatured.unique()) # artists + collabrations

In [ ]:
df_nocollaboration = df[df.iscollab==0]

the following analysis are all done within songs w/o collaboration

## the single artists who land top 1 the most(w/o collaboration)

In [ ]:
df_land1 = df_nocollaboration .loc[(df_nocollaboration ['weeks-on-board'] == 1)&(df_nocollaboration ['peak-rank']==1)]
artist_land1_count = df_land1.groupby('artist_nofeatured')['song'].nunique().reset_index()
artist_land1_count.sort_values(by="song",ascending=False)

## the artist who have most songs on billboard

In [ ]:
num_on_billboard = df_nocollaboration.groupby('artist_nofeatured')['song'].nunique().reset_index()

In [ ]:
num_on_billboard = num_on_billboard.sort_values(by="song",ascending=False).set_index('artist_nofeatured')

In [ ]:
plt.figure()
num_on_billboard.head(15).plot.barh(y = 'song',edgecolor = "none")
plt.gca().invert_yaxis()
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1), frameon=False)

only top 15 singer are shown in the bar graph

## every artist peak rank mean and sd

In [ ]:
artists = df_nocollaboration.groupby(['artist_nofeatured','song'],sort=False)['peak-rank'].max().reset_index()
count = lambda x: len(x.unique())
artists_peak = pd.pivot_table(artists ,index=['artist_nofeatured'],values= 'peak-rank',aggfunc=[np.mean,np.std,count])
artists_peak = artists_peak.sort_values(by=[('<lambda>', 'peak-rank'),('mean', 'peak-rank')],ascending = [False,True]).reset_index(level=[0])
artists_peak.columns=['artist','average peak rank','stand deviation of peak rank','songs on billboard']
artists_peak.head(50)

seems like the collaboration isn't cleaned very well.... like "With"...